In [1]:
#Inisialisasi Library

from PIL import Image
from PIL import ImageDraw

import glob
import pandas as pd
from skimage.feature import greycomatrix, greycoprops
from skimage.transform import resize
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import math


Bad key "text.kerning_factor" on line 4 in
C:\Users\ASUS\Anaconda3\envs\tensorflowproject\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
ubah8bit_3bit = [i // 32 for i in range(256)]

In [3]:
# Fungsi-fungsi
# Proses membuka gambar

def read_file(mypath):
    file_name = []
    label = []
    full_path = []

    for path, subdirs, files in os.walk(mypath):
        for name in files:
            full_path.append(os.path.join(path, name))
            label.append(path.split('/')[-1])
            
    return full_path, label

def load_img(img_path, dim):
    img = Image.open(img_path).convert('L')
    hasil_img = img.resize([dim, dim], Image.NEAREST)
    return hasil_img

def print_img(img):
    plt.imshow(img, cmap='gray')
    
def getLabels(path):
    label = os.listdir(path)
    return label
    
# Preprocessing Gambar
    
def getAllColorIMG(img):
    AllColorIMG = list(img.getdata())
    return AllColorIMG

def ubahKe3bit(AllColorIMG):
    i = 0
    
    for ColorIMG in AllColorIMG:
        AllColorIMG[i] = ubah8bit_3bit[ColorIMG]
        i += 1
        
    return AllColorIMG

def ubahKeMatriks(AllColorIMG):
    matriksIMG = [] 
    i = 0
    
    dimensiGambar = int(math.sqrt(len(AllColorIMG)))
    
    for row in range(dimensiGambar):
        newRowMat = []
        for col in range(dimensiGambar):
            newRowMat.append(AllColorIMG[i])
            i += 1
        matriksIMG.append(newRowMat)
        
    return matriksIMG

def featureExtract(img):
    colorIMG = getAllColorIMG(img)
    _3BitColor = ubahKe3bit(colorIMG)
    matriksIMG = ubahKeMatriks(_3BitColor)
    
    resultOfGLCM = []
    glcm = greycomatrix(matriksIMG, distances=[2], angles=[45], levels=256, symmetric=False, normed=False)
    
    komponen = ['dissimilarity', 'contrast', 'homogeneity', 'energy', 'correlation', 'ASM']
    
    for i in komponen:
        resultOfGLCM.append(greycoprops(glcm, i)[0][0])
    
    return resultOfGLCM

In [4]:
# Memproses File Training dengan mengekstrak fiturnya sebelum dimodelkan

train_path = 'dataset_magang/Training/'
imgType = '*.jpg'
dim = 224

PathOf_trainIMG = []
LabelOf_trainIMG = []
Dissimilarity = []
Contrast = []
Homogeneity = []
Energy = []
Correlation = []
ASM = []

labels = getLabels(train_path)

for label in labels:
    full_paths = glob.glob(train_path + label + '/' + imgType)
    for full_path in full_paths:
        PathOf_trainIMG.append(full_path)
        LabelOf_trainIMG.append(label)
        trainIMG = load_img(full_path, dim)
        featureOfIMG = featureExtract(trainIMG)
        Dissimilarity.append(featureOfIMG[0])
        Contrast.append(featureOfIMG[1])
        Homogeneity.append(featureOfIMG[2])
        Energy.append(featureOfIMG[3])
        Correlation.append(featureOfIMG[4])
        ASM.append(featureOfIMG[5])

columns = ["Path", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM" , "Label"]
variables = {"Path":PathOf_trainIMG, "Dissimilarity":Dissimilarity, "Contrast":Contrast, "Homogeneity":Homogeneity, "Energy":Energy, "Correlation":Correlation, "ASM":ASM ,"Label":LabelOf_trainIMG}
df_train = pd.DataFrame(variables, columns = columns)
df_train.to_csv('DataBatik.csv', index=False) 

In [5]:
# Memproses File Validasi dengan mengekstrak fiturnya sebelum dimodelkan

val_path = 'dataset_magang/Validasi/'
imgType = '*.jpg'
dim = 224

PathOf_trainIMG = []
LabelOf_trainIMG = []
Dissimilarity = []
Contrast = []
Homogeneity = []
Energy = []
Correlation = []
ASM = []

labels = getLabels(val_path)

for label in labels:
    full_paths = glob.glob(train_path + label + '/' + imgType)
    for full_path in full_paths:
        PathOf_trainIMG.append(full_path)
        LabelOf_trainIMG.append(label)
        trainIMG = load_img(full_path, dim)
        featureOfIMG = featureExtract(trainIMG)
        Dissimilarity.append(featureOfIMG[0])
        Contrast.append(featureOfIMG[1])
        Homogeneity.append(featureOfIMG[2])
        Energy.append(featureOfIMG[3])
        Correlation.append(featureOfIMG[4])
        ASM.append(featureOfIMG[5])

columns = ["Path", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM" , "Label"]
variables = {"Path":PathOf_trainIMG, "Dissimilarity":Dissimilarity, "Contrast":Contrast, "Homogeneity":Homogeneity, "Energy":Energy, "Correlation":Correlation, "ASM":ASM ,"Label":LabelOf_trainIMG}
df_val = pd.DataFrame(variables, columns = columns)
df_val.to_csv('DataValidasiBatik.csv', index=False) 